# Sentiment Analysis

In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 81.6 MB/s eta 0:00:00m eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 13.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.9 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 41.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 37.3 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=994148f64998e17d085808d98374dab436e1b06675c751a1ff30acf832f0e8eb
  Stored in dir

In [2]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [3]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


La base de datos cuenta con 891 observaciones y con 3 variables, de las cuales dos están clasificadas como objetos y una como entera.

In [5]:
df.sample(10)

,package_name,review,polarity
495,com.Slack,back to greatness previously couldn't compose...,1
517,com.dropbox.android,continually fails to upload files great when ...,0
227,com.supercell.clashofclans,not a fan of new update new update makes it d...,0
858,com.rovio.angrybirds,great fun. very addictive. still brilliant! l...,1
710,com.opera.mini.native,saved pages list should be in alphabetical or...,0
344,com.viber.voip,"wtf keeps going offline automatically ,my mes...",0
364,com.google.android.talk,really? i honestly ussd to love this app.... ...,0
473,com.whatsapp,not working at all after updating..hangs pho...,0
123,com.king.candycrushsaga,either make the moving forward process 3 bars...,0
414,com.facebook.orca,we have wifi with full internet connection. ...,0


Se observa que las dos primeras variables deberían ser string mientras que la última debería ser categórica, por lo que se procede a hacer las transformaciones.

In [13]:
df[df.select_dtypes('object').columns]=df[df.select_dtypes('object').columns].astype('string')
df['polarity']=df['polarity'].astype('category')

A continuación se eliminan algunos caracteres desconocidos que se detectaron en el texto.

In [6]:
df['review']=df['review'].str.strip()
df['review']=df['review'].str.lower()

In [7]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)


NameError: name 'normalize_str' is not defined

In [ ]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [ ]:
df.sample(10)

,package_name,review,polarity
332,com.viber.voip,cant use not being able to call any contact pl...,0
502,com.Slack,great internal comms system for our business,1
140,com.king.candycrushsaga,was great i was going to give this 5 stars unt...,0
829,com.hamropatro,j khojyo thyehi paine ek matra apps,1
39,com.facebook.katana,app doesn't work after latest upgrade the face...,0
575,jabanaki.todo.todoly,i love this guy. he keep updating this simple ...,1
23,com.facebook.katana,i wish posts popped up in the order they were ...,0
200,com.supercell.clashofclans,force close it's been 1 or 2 months and y'all ...,0
438,com.whatsapp,no notification displayed on system notificati...,0
277,com.android.chrome,translation use it for browsing and visiting v...,0


In [ ]:
df.iloc[675,]

package_name                                    com.hamrokeyboard
review          loved it rzrl app rrlrrl rrsrzrlrrl r r rsrzrl...
polarity                                                        1
Name: 675, dtype: object

In [ ]:
X=df['review']
y=df['polarity']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec=CountVectorizer(stop_words='english')
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()